In [1]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

from xgboost import XGBClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

In [2]:
''' Seed '''
seed_everything(42)

''' Data Load '''
train, test, sample_submission = load_data()

''' Train / Vaild Split'''
X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=42)

X_train.drop(columns = ['ID'], inplace = True)
X_valid.drop(columns = ['ID'], inplace = True)
numeric_col = X_train.select_dtypes(include=['float64', 'float32', 'int64', 'int32', 'int16', 'int8']).columns
object_col = X_train.select_dtypes(include=['object']).columns

''' 빈도 인코딩 '''
print('Start Frequency')
encoder =  ce.CountEncoder()
X_train[object_col] = encoder.fit_transform(X_train[object_col], y_train)
X_valid[object_col] = encoder.transform(X_valid[object_col], y_valid)
test[object_col] = encoder.transform(test[object_col])

Start Frequency


In [3]:
''' F17 '''
X_train, X_valid, test = groupby_mean(X_train, X_valid, test, 'F17', X_train.columns)
X_train.head()

100%|██████████| 39/39 [00:21<00:00,  1.78it/s]


,F01,F02,F03,F04,F05,F06,F07,F08,F09,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F17_F01_mean,F17_F02_mean,F17_F03_mean,F17_F04_mean,F17_F05_mean,F17_F06_mean,F17_F07_mean,F17_F08_mean,F17_F09_mean,F17_F10_mean,F17_F11_mean,F17_F12_mean,F17_F13_mean,F17_F14_mean,F17_F15_mean,F17_F16_mean,F17_F17_mean,F17_F18_mean,F17_F19_mean,F17_F20_mean,F17_F21_mean,F17_F22_mean,F17_F23_mean,F17_F24_mean,F17_F25_mean,F17_F26_mean,F17_F27_mean,F17_F28_mean,F17_F29_mean,F17_F30_mean,F17_F31_mean,F17_F32_mean,F17_F33_mean,F17_F34_mean,F17_F35_mean,F17_F36_mean,F17_F37_mean,F17_F38_mean,F17_F39_mean
12066756,24,39,10216484,46.0,24,0,1033,10108558,5812,24,514.0,24,9051,20,9548796,7094,10279461,2.0,1.0,27707,8191097,7577062,4311788,6.0,7094,10062514,11.0,990402,5.0,1190,14162781,0.0,1.0,24,22758893,1.0,9051,0.0,7094,1.420193e+06,1.512032e+06,7.604958e+06,27.464969,1.417702e+06,120.609255,537185.312380,5.998524e+06,136637.103254,1.430448e+06,422.610359,1.420606e+06,569794.905333,10.511455,7.856585e+06,30949.504543,10279461.0,7.152972,0.394988,4.565390e+06,4.656303e+06,3.010935e+06,7.455932e+06,107.256788,47231.648266,7.391514e+06,18.297700,6.677864e+06,3.841584,1.624944e+06,9.380930e+06,15909.988752,2.589174,1.421212e+06,2.075793e+07,8.363200,221443.291706,0.000060,58414.415208
5255865,2211867,2369136,9490820,NaN,2211863,4,60483,5891923,233449,2211867,13.0,2211867,1635381,3,9490820,269653,1511983,3.0,0.0,9490820,8191097,7577062,12862549,7.0,227635,9490820,7.0,971069,1.0,2526,14162781,5152.0,0.0,2211867,22758893,3.0,238589,0.0,558317,1.649268e+06,1.747222e+06,7.894761e+06,25.311914,1.646995e+06,122.692317,604520.264766,6.145985e+06,170049.357413,1.657322e+06,525.531200,1.649676e+06,600684.674135,11.015871,8.057470e+06,45411.301235,1511983.0,7.278606,0.390564,4.584904e+06,4.779708e+06,3.016495e+06,7.457931e+06,129.057189,66758.659202,7.675641e+06,40.817297,6.908544e+06,6.884567,1.627874e+06,8.560414e+06,13027.534981,2.926691,1.650169e+06,2.092484e+07,8.804750,252763.187605,0.410877,98974.935599
13191452,1111326,1111326,1919845,5.0,1111326,29,2270,5891923,8186,1111326,266.0,1111326,25173,4,2797159,3497,10279461,6.0,1.0,436781,8191097,96765,12862549,4.0,3539,1628146,1.0,12085685,1.0,377455,1193003,23.0,3.0,1111326,22758893,0.0,25173,0.0,3539,1.420193e+06,1.512032e+06,7.604958e+06,27.464969,1.417702e+06,120.609255,537185.312380,5.998524e+06,136637.103254,1.430448e+06,422.610359,1.420606e+06,569794.905333,10.511455,7.856585e+06,30949.504543,10279461.0,7.152972,0.394988,4.565390e+06,4.656303e+06,3.010935e+06,7.455932e+06,107.256788,47231.648266,7.391514e+06,18.297700,6.677864e+06,3.841584,1.624944e+06,9.380930e+06,15909.988752,2.589174,1.421212e+06,2.075793e+07,8.363200,221443.291706,0.000060,58414.415208
24944338,2211867,2369136,9490820,NaN,2211863,656,10499,951379,7501,2211867,163.0,2211867,736398,5,9490820,4231,3169397,1.0,0.0,9490820,8191097,3330771,12862549,617.0,10421,9490820,1.0,2418224,1.0,116,14162781,33685.0,0.0,2211867,2985020,1.0,7501,0.0,10445,1.247917e+06,1.346923e+06,7.854655e+06,38.018995,1.245862e+06,113.150739,916599.169426,6.060678e+06,123803.848699,1.262386e+06,279.480631,1.248390e+06,583245.655836,7.273670,7.959318e+06,22498.358113,3169397.0,6.102364,0.306398,4.592436e+06,4.737174e+06,3.015036e+06,7.454161e+06,119.845870,34507.436376,7.642485e+06,1.563494,6.700548e+06,1.095607,1.626655e+06,9.626691e+06,28289.718768,0.417189,1.249172e+06,2.015171e+07,7.581805,202987.629839,0.000033,39743.718700
9685226,166,155841,9490820,NaN,1,1,49354,5891923,1857,9912,183.0,781,8810,0,9490820,6225,3557269,NaN,NaN,9490820,1241173,7577062,12862549,NaN,10931,9490820,NaN,12085685,NaN,1611900,14162781,80686.0,NaN,2743,2985020,NaN,2409,0.0,10931,1.304924e+06,1.400804e+06,7.704502e+06,30.991392,1.302585e+06,115.572194,743766.086707,6.020089e+06,134021.128390,1.317241e+06,304.993638,1.305368e+06,571165.251696,8.336569,7.873246e+06,25288.062295,3557269.0,6.572688,0.319805,4.443621e+06,

In [4]:
''' 결측값 '''
X_train = X_train.fillna(0)
X_valid = X_valid.fillna(0)
test = test.fillna(0)

''' 메모리 '''
X_train = reduce_mem_usage(X_train)
X_valid = reduce_mem_usage(X_valid)
test = reduce_mem_usage(test)

Memory usage of dataframe is 15516.99 MB
Memory usage after optimization is: 6776.41 MB
Decreased by 56.3%
Memory usage of dataframe is 1724.11 MB
Memory usage after optimization is: 752.93 MB
Decreased by 56.3%
Memory usage of dataframe is 2735.48 MB
Memory usage after optimization is: 1185.95 MB
Decreased by 56.6%


In [7]:
X_train.to_csv('Count__X_train.csv', index=False)
X_valid.to_csv('Count_X_valid.csv', index=False)
y_train.to_csv('Count_train.csv', index=False)
y_valid.to_csv('Count_valid.csv', index=False)
test.to_csv('Count_test.csv', index=False)

In [2]:
X_train = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/Count__X_train.csv') 
X_valid = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/Count_X_valid.csv')
y_train = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/Count_train.csv')
y_valid = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/Count_valid.csv')
test = pd.read_csv('/home/workspace/DACON/Click_predict/Model/LGBM_Experiment/Count_test.csv')

In [7]:
def objective(trial, t_X, t_y, v_X, v_y):
  params = {
        "tree_method": 'gpu_hist',
        "max_depth": trial.suggest_int("max_depth", 10, 40),
        "learning_rate": trial.suggest_float('learning_rate', 0.001, 0.2),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 4000),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 0.1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0),
        "random_state": 42,
    }

  model = XGBClassifier(**params)
  model.fit(t_X, t_y, eval_metric='auc')
  pred = model.predict_proba(v_X)
  score = roc_auc_score(v_y, pred[:, 1])

  return score

In [8]:
study = optuna.create_study(study_name='XCClassifier', direction='maximize', sampler=TPESampler(seed=42))
study.optimize(lambda trial: objective(trial, X_train, y_train, X_valid, y_valid), n_trials=10)

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2024-06-02 08:53:33,972] A new study created in memory with name: XCClassifier


[I 2024-06-02 09:34:51,625] Trial 0 finished with value: 0.7441644745437822 and parameters: {'max_depth': 21, 'learning_rate': 0.19019214697557332, 'n_estimators': 3196, 'colsample_bytree': 0.7993292420985183, 'colsample_bylevel': 0.5780093202212182, 'colsample_bynode': 0.5779972601681014, 'reg_lambda': 0.006750277604651747, 'reg_alpha': 0.8663099696291603, 'subsample': 0.7207805082202461, 'min_child_weight': 36, 'gamma': 0.1185260448662222}. Best is trial 0 with value: 0.7441644745437822.
[I 2024-06-02 10:22:46,404] Trial 1 finished with value: 0.7365445028293475 and parameters: {'max_depth': 40, 'learning_rate': 0.16665608551928393, 'n_estimators': 1637, 'colsample_bytree': 0.5909124836035503, 'colsample_bylevel': 0.5917022549267169, 'colsample_bynode': 0.6521211214797689, 'reg_lambda': 0.05295088673159155, 'reg_alpha': 0.43251307362347363, 'subsample': 0.5038603981386294, 'min_child_weight': 31, 'gamma': 0.22554447458683766}. Best is trial 0 with value: 0.7441644745437822.
[I 2024-0

: 